In [1]:
import pandas as pd
import os
import subprocess

import jpype
import jpype.imports
from jpype import startJVM, shutdownJVM, java, addClassPath, JClass

In [ ]:
# TODO: Implement uniform testing (not exponential), think about using min, max, median, and mean when getting times for a particular array length

In [2]:
startJVM(convertStrings=False, classpath=["../out/production/SortComplexity"])

In [3]:
# java.lang.System.out.println("Hello world !")
try:
    benchmark_class = JClass("stats.Benchmarks")
    print(benchmark_class.averageMergeSortTime(1000, 256))
except Exception as e:
    print(f"Exception: {e}")

37269


In [4]:
def reset_file(filename, reset_text):
    with open(filename, "w") as f:
        f.write(reset_text)

In [5]:
def run_exponential_sizes(start_size, base, x, n):
    df = pd.DataFrame(columns=["size", "merge", "selection"])
    for i in range(x+1):
        size = start_size + base**i
        merge_time = benchmark_class.averageMergeSortTime(n, size)
        selection_time = benchmark_class.averageSelectionSortTime(n, size)
        
        df.loc[len(df.index)] = [size, merge_time, selection_time]
        
    return df
    
    # os.system(f"java -cp ../out/production/SortComplexity stats.Benchmarks {func} {n} {start_size + base ** i} ../analysis/{func}_times.csv")

In [6]:
def find_crossover_point(base, x, n, start_size=0):
    while True:
        
        times = run_exponential_sizes(start_size, base, x, n)
        # initial_merge = pd.read_csv("merge_times.csv")
        
        # run_exponential_sizes(start_size, base, x, n, "selection")
        # initial_selec = pd.read_csv("selection_times.csv")
        
        # reset_file("merge_times.csv", "size,time\n")
        # reset_file("selection_times.csv", "size,time\n")

        # times = initial_merge.merge(initial_selec, on=["size"], suffixes=["_merge", "_selection"])
        # times.rename(columns={"time_merge": "merge", "time_selection": "selection"}, inplace=True)
        
        # display(times)
        
        merge_faster = times[["size", "merge"]].rename(columns={"merge": "faster"}).copy()
        merge_faster["faster"] = times["merge"] < times["selection"]
        # faster["selection"] = ~faster["merge"]
        
        if merge_faster.loc[0, "faster"]:
            return merge_faster.loc[0, "size"]
        
        # display(merge_faster)
        
        col = merge_faster.faster.to_numpy()
        if not (col[0] == col).all():
            last_slower_index = merge_faster[~merge_faster["faster"]].index[-1]
            if last_slower_index+1 >= merge_faster.shape[0]:
                continue
            
            last_slower_size = merge_faster.loc[last_slower_index, "size"]
            first_faster_size = merge_faster.loc[last_slower_index+1, "size"]
            
            break
    
    # display(last_slower_index)
    # display(last_slower_size)
    
    if first_faster_size - last_slower_size == 1:
        return first_faster_size
    
    return find_crossover_point(base, last_slower_index, n, start_size=last_slower_size)
    

In [7]:
sample_size = 20
sizes = []
for i in range(sample_size):
    sizes.append(find_crossover_point(2, 10, 1000))
    
sum(sizes) / len(sizes)
# run_exponential_sizes(32, 2, 5, 100, "merge")

57.7

In [9]:
sizes

[1, 61, 61, 59, 56, 55, 60, 55, 65, 59, 56, 65, 59, 61, 60, 61, 65, 65, 65, 65]

In [8]:
shutdownJVM()